In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
import analysis.lib.purification.purify_analysis_params as analysis_params;reload(analysis_params)
%matplotlib inline

folder_lt3 = r'D:\measuring\data\Purification_lt3_raw_data\Eta_vs_Theta'
folder_lt4 = r'D:\measuring\data\Purification_lt4_raw_data\Eta_vs_Theta'

### ssro calibrations
day = '20160726_' # for ssro calibration

ssro_calib_lt3 = day+'095123'
ssro_calib_lt4 = day+'104600'

Purify_analysis = pa.purify_analysis('Eta_v_Theta',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

### list of good timestamps and RO calibration
all_lt3,all_lt4 = [],[]

### get all valid time stamps. remember that data has to be stored locally.
msmt_days = ['20160726','20160824']
theta_percent = 35

for d in msmt_days:
    tstamp_lt3,tstamp_lt4 = Purify_analysis.tstamps_for_both_setups(d,contains = 'ZZ_'+str(theta_percent)+'percent')
                                                                   # ,newest_tstamp = '133400') ### newest timestamp allows for only taking parts of a day.
    all_lt3.extend(tstamp_lt3)
    all_lt4.extend(tstamp_lt4)

Purify_analysis.load_raw_data(all_lt3,all_lt4,verbose = True)

offset = analysis_params.data_settings['shifted_data_correction_time']*np.ones(np.shape(all_lt4))
offset_ch1 = analysis_params.data_settings['shifted_data_start_offset_ch1']*np.ones(np.shape(all_lt4))
Purify_analysis.correct_pq_times(offsets=offset,offsets_ch1=offset_ch1)
print all_lt3,all_lt4

In [ ]:
##### Set filters:
photon_channel = 2 # 0 or 1; 2 means both HH detectors
st_start = 3055e3#2763e3
st_len       = 40e3 #50 ns
st_len_w2    = 40e3
ch1_offset = 0
max_reps_w2 = 250

Purify_analysis.apply_temporal_filters_to_prefiltered_data(st_start = st_start,
                                                       st_len = st_len,
                                                       st_len_w2 = st_len_w2
                                                       )
Purify_analysis.apply_sync_filter_w1_w2()
Purify_analysis.correlate_RO_results_no_purification(apply_ROC = True)

In [ ]:
### this window can be used to analyze one of the full timetrace in order to get a feeling for the tail and the windows.
### later on all timestamps should be combined to get a clear pikkcture

a_lt4 = ppq.purifyPQAnalysis(tb.data_from_time(all_lt4[0],folder = folder_lt4))

vert0 = st_start/1000. ### start of the first window

# BK_analysis.plot_total_histogram(1,st_start-20e3,200e3,1e3,log_plot=True)
pq_plots.plot_marker_filter_comparison(a_lt4.pqf,mrkr_chan = 1,start =3045e3,  length = 80e3, hist_binsize = 0.1e3, save = False, log=False,plot_threshold_ch0 =vert0,plot_threshold_ch1 =vert0) #start =7480

In [ ]:
#### compare theory vs. reality
theta_vals = [1/3.,0.25,1/5.,1/6.,0.125,0.1] #in units of pi
eta_vals = [0.79,0.51,0.3959,0.29,0.18,0.14]

a = [ 0.06983436,  0.21139007,  0.33169605,  0.38707952]
b =[ 0.00516695,  0.00663856 , 0.00776175,  0.00842587]

real_theta_vals = 0.5-np.array([0.153355,0.2525,0.2827,0.323754,0.362197,0.3865])
def eta_theory(theta,Ploss):
    return (np.sin(theta*np.pi)**2*(1+Ploss))/(2-(1-Ploss)*np.sin(theta*np.pi)**2) 
print eta_theory(1/3.,0.99)
fig = plt.figure()
ax = plt.subplot()
theory_range = np.linspace(0,0.5,100)
plt.plot(theory_range,eta_theory(theory_range,0.99),label = 'Theory')
plt.plot(theta_vals,eta_vals,marker = 'o',label='Naive data')
plt.plot(real_theta_vals,eta_vals,'o',label='theta from SPCorrs')
plt.legend(loc=2)

ax.set_xlabel(r'$\theta$ ($\pi$)',fontsize=18)
ax.set_ylabel(r'$\eta$',fontsize=18)
plt.show()
plt.close('all')

In [ ]:
def format_plot(fig,ax):
    linewidths = 1.5
    textsize = 14
    ax.tick_params(labelsize = textsize,width =linewidths)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(linewidths)
    ax.yaxis.label.set_size(textsize)
    ax.xaxis.label.set_size(textsize)
    
    
def num2str(num, precision): 
    return "%0.*f" % (precision, num)

save_plot_folder = r'D:\measuring\data\purification_plots'
def save_and_close_plot(f = save_plot_folder, save = False, name = None):

    if name == None:
        name = 'Results'

    if save:
        plt.savefig(os.path.join(f,name+'.pdf'),format='pdf',bbox_inches='tight')
        plt.savefig(os.path.join(f,name+'.png'),format='png',bbox_inches='tight')

    plt.show()
    plt.close('all')
    
def plot_parity_results(corr,err=np.array([0,0]),events = np.array([0,0]),state = 'parity_mixed'):


    fig = plt.figure()
    ax = plt.subplot()
    plt.ylabel('Expectation value')
    # plt.title('Corrected')
    plt.bar(np.arange(0,2), np.squeeze(np.asarray(corr)), yerr=err, align = 'center',
            color = '#436CDE',linewidth = 1.5,error_kw=dict(lw = 1.5,capthick=1.5))
    for k in range(2):
        plt.text(k,0.02, num2str(np.squeeze(np.asarray(corr))[k],2), ha = 'center')
#         plt.text(k,err[k]+corr[k]+0.02, num2str(np.squeeze(np.asarray(events))[k],0), ha = 'center')

    if np.amax(corr)+np.amax(err) > 0.5:
        plt.ylim([0,np.amax(corr)+np.amax(err)])
    else:
        plt.ylim([-0.7,0.7])
    plt.xticks([0,1], ['<XX>','<ZZ>'])
    ax.set_xlim(-0.6,1.6)
    format_plot(fig,ax)
    plt.axhline(0,lw=1.5,color = 'black')
    ax.tick_params(top = 'off',bottom = 'off')
    save_and_close_plot(save = True,name = 'Separable_state')
    


In [ ]:
### ZZ probabiliteis values:
ZZ_p = [ 0.00347118,  0.21944022,  0.48764476,  0.28944384]
ZZ_p_u = [ 0.00597591,  0.00708,     0.00996275,  0.00922813]


### XX probabilities values:
XX_p = [ 0.19011589,  0.29882374,  0.22117005,  0.28989031]
XX_p_u = [ 0.00778677,  0.00888261,  0.00793844,  0.00881881]

### calculate expectation values
ZZ = (ZZ_p[0]+ZZ_p[3]-0.5)*2
ZZ_u = np.sqrt(ZZ_p_u[0]**2+ZZ_p_u[3]**2)*2
XX = (XX_p[0]+XX_p[3]-0.5)*2
XX_u = np.sqrt(XX_p_u[0]**2+XX_p_u[3]**2)*2

plot_parity_results(np.array([XX,ZZ]),err = np.array([XX_u,ZZ_u]))